# Singapore HDB Resale Price Prediction

In [3]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score 

%matplotlib inline

In [4]:
# Define all required functions here
def normalization (x, min, max):
    normalize = (x - min)/(max - min)
    return float(normalize)

def convert_lease (x):
    array_x = x.split(sep=" ")
    if (len(array_x) > 2):
        month = int(array_x[2])/12
        x_convert = int(array_x[0]) + month
        return float(x_convert)
    else:
        x_convert = array_x[0]
        return float(x_convert)

def convert_flattype (x):
    if "2 ROOM" in x:
        return int(0)
    elif "3 ROOM" in x:
        return int(1)
    elif "4 ROOM" in x:
        return int(2)
    elif "5 ROOM" in x:
        return int(3)
    elif "EXECUTIVE" in x:
        return int(4)
    else:
        return int(5)

def convert_storeyrange (x):
    if "01 TO 03" in x:
        return int(0)
    elif "04 TO 06" in x:
        return int(1)
    elif "07 TO 09" in x:
        return int(2)
    elif "10 TO 12" in x:
        return int(3)
    elif "13 TO 15" in x:
        return int(4)
    else:
        return int(5)

In [5]:
# Reading CSVs into dataframe
hdb_df = pd.read_csv("all_geocoded.csv")
print (hdb_df.shape)
hdb_df.head()

(57386, 18)


,month,town,flat_type,addresses,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,searchval,X,Y,lat,long,distanceWithMrt,distanceWithRaffles,distanceWithGdPri
0,2018-01,ANG MO KIO,2 ROOM,314 ANG MO KIO AVE 3,10 TO 12,44.0,Improved,1978,59 years 01 month,250000.0,TECK GHEE EVERGREEN,29865.99805,38695.97027,1.366227,103.850086,416.686149,9162.186778,1396.420210
1,2018-01,ANG MO KIO,3 ROOM,109 ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,59 years 07 months,240000.0,KEBUN BARU HEIGHTS,28486.23758,39123.83010,1.370097,103.837688,1321.143682,9713.392312,552.239946
2,2018-01,ANG MO KIO,3 ROOM,150 ANG MO KIO AVE 5,01 TO 03,68.0,New Generation,1981,62 years,240000.0,YIO CHU KANG GROVE,28968.13327,39865.81646,1.376807,103.842018,640.152689,10391.930070,879.171242
3,2018-01,ANG MO KIO,3 ROOM,559 ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1980,61 years,242000.0,CHENG SAN PLACE,30730.76021,39223.53457,1.370998,103.857856,931.080675,9718.096454,1885.848306
4,2018-01,ANG MO KIO,3 ROOM,461 ANG MO KIO AVE 10,10 TO 12,68.0,New Generation,1980,61 years 01 month,255000.0,HDB-ANG MO KIO,30681.04009,38839.80113,1.367528,103.857410,913.873510,9329.200120,1984.578623


In [6]:
# Converting non-numerics columns to numerics
hdb_df['flat_type_num'] = hdb_df['flat_type'].apply(convert_flattype)
hdb_df['remaining_lease_num'] = hdb_df['remaining_lease'].apply(convert_lease)
hdb_df['storey_range_num'] = hdb_df['storey_range'].apply(convert_storeyrange)
hdb_df.shape

(57386, 21)

In [5]:
# Identifying the columns available in dataframe
hdb_df.columns

Index(['month', 'town', 'flat_type', 'addresses', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date',
       'remaining_lease', 'resale_price', 'searchval', 'X', 'Y', 'lat', 'long',
       'distanceWithMrt', 'distanceWithRaffles', 'distanceWithGdPri',
       'flat_type_num', 'remaining_lease_num', 'storey_range_num'],
      dtype='object')

In [7]:
hdb_df.head()

,month,town,flat_type,addresses,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,...,X,Y,lat,long,distanceWithMrt,distanceWithRaffles,distanceWithGdPri,flat_type_num,remaining_lease_num,storey_range_num
0,2018-01,ANG MO KIO,2 ROOM,314 ANG MO KIO AVE 3,10 TO 12,44.0,Improved,1978,59 years 01 month,250000.0,...,29865.99805,38695.97027,1.366227,103.850086,416.686149,9162.186778,1396.420210,0,59.083333,3
1,2018-01,ANG MO KIO,3 ROOM,109 ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,59 years 07 months,240000.0,...,28486.23758,39123.83010,1.370097,103.837688,1321.143682,9713.392312,552.239946,1,59.583333,0
2,2018-01,ANG MO KIO,3 ROOM,150 ANG MO KIO AVE 5,01 TO 03,68.0,New Generation,1981,62 years,240000.0,...,28968.13327,39865.81646,1.376807,103.842018,640.152689,10391.930070,879.171242,1,62.000000,0
3,2018-01,ANG MO KIO,3 ROOM,559 ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1980,61 years,242000.0,...,30730.76021,39223.53457,1.370998,103.857856,931.080675,9718.096454,1885.848306,1,61.000000,0
4,2018-01,ANG MO KIO,3 ROOM,461 ANG MO KIO AVE 10,10 TO 12,68.0,New Generation,1980,61 years 01 month,255000.0,...,30681.04009,38839.80113,1.367528,103.857410,913.873510,9329.200120,1984.578623,1,61.083333,3


In [8]:
# Re-order dataframe based on different flat type
flat_type = ['2 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', 'EXECUTIVE', 'MULTI-GENERATION']
all_hdb_flat_df = []

for flat in flat_type:
    temp_df = hdb_df[hdb_df['flat_type'] == flat]
    all_hdb_flat_df.append(temp_df)

In [9]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

with pd.ExcelWriter("hdb_numerics_corr_matrix.xlsx") as writer:
    for i in range(0,6):
        hdb_numerics_df = all_hdb_flat_df[i].select_dtypes(include=numerics)
        hdb_numerics_df_corr = hdb_numerics_df.corr()
        hdb_numerics_df_corr.style.background_gradient(cmap='coolwarm').set_precision(2).to_excel(writer, sheet_name=flat_type[i])

In [10]:
group_multiple = hdb_df.groupby(['town', 'flat_type']).agg({'resale_price': ['mean', 'min', 'max']})
group_multiple.to_csv("hdb_group.csv")
group_multiple

resale_price                     
                                      mean       min        max
town       flat_type                                           
ANG MO KIO 2 ROOM            203333.333333  178000.0   250000.0
           3 ROOM            288837.094177  174000.0   519000.0
           4 ROOM            467219.239193  275000.0   782000.0
           5 ROOM            674743.531863  420000.0   970888.0
           EXECUTIVE         811086.750000  623000.0  1030000.0
...                                    ...       ...        ...
YISHUN     3 ROOM            272516.216667  205000.0   448000.0
           4 ROOM            359630.018447  240000.0   625000.0
           5 ROOM            471712.538141  330000.0   758000.0
           EXECUTIVE         578406.892523  450000.0   788888.0
           MULTI-GENERATION  765245.473684  660000.0   868000.0

[128 rows x 3 columns]

In [11]:
# Check the data-type for each columns
all_hdb_flat_df[0].dtypes

month                   object
town                    object
flat_type               object
addresses               object
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
remaining_lease         object
resale_price           float64
searchval               object
X                      float64
Y                      float64
lat                    float64
long                   float64
distanceWithMrt        float64
distanceWithRaffles    float64
distanceWithGdPri      float64
flat_type_num            int64
remaining_lease_num    float64
storey_range_num         int64
dtype: object

In [12]:
hdb_features_df = []

for j in range (0, 6):
    temp_features_df = all_hdb_flat_df[j][['floor_area_sqm', 
                                           'X', 
                                           'Y', 
                                           'distanceWithMrt', 
                                           'distanceWithRaffles', 
                                           'distanceWithGdPri', 
                                           'flat_type_num', 
                                           'remaining_lease_num', 
                                           'storey_range_num', 
                                           'resale_price']]
    hdb_features_df.append(temp_features_df)
    
hdb_features_df

[       floor_area_sqm            X            Y  distanceWithMrt  \
 0                44.0  29865.99805  38695.97027       416.686149   
 55               45.0  38182.29501  35026.16233       783.869194   
 168              34.0  27839.11989  29967.31053       510.157427   
 169              45.0  25355.90288  28249.96848       119.075919   
 255              53.0  28677.59670  29977.19741       559.457438   
 ...               ...          ...          ...              ...   
 57012            43.0  30132.68196  35602.31567       679.007626   
 57071            47.0  24419.97266  47713.39739       800.440329   
 57072            47.0  24419.97266  47713.39739       800.440329   
 57073            47.0  24405.26496  47842.23908       929.960779   
 57222            47.0  29639.31594  44998.66965      1608.152888   
 
        distanceWithRaffles  distanceWithGdPri  flat_type_num  \
 0              9162.186778        1396.420210              0   
 55             9824.205002         809.

In [13]:
# Normalizing features columns
hdb_data_model_df = []

flrarea_min = []
flrarea_max = []
x_min = []
x_max = []
y_min = []
y_max = []
distmrt_min = []
distmrt_max = []
distraffles_min = []
distraffles_max = []
distgdpri_min = []
distgdpri_max = []
rlease_min = []
rlease_max = []
storeyrng_min = []
storeyrng_max = []
rprice_min = []
rprice_max = []

for k in range(0, 6):
    temp = hdb_features_df[k]['floor_area_sqm'].min()
    flrarea_min.append(temp)
    temp = hdb_features_df[k]['floor_area_sqm'].max()
    flrarea_max.append(temp) 
    temp = hdb_features_df[k]['X'].min()
    x_min.append(temp)
    temp = hdb_features_df[k]['X'].max()
    x_max.append(temp)
    temp = hdb_features_df[k]['Y'].min()
    y_min.append(temp)
    temp = hdb_features_df[k]['Y'].max()
    y_max.append(temp)
    temp = hdb_features_df[k]['distanceWithMrt'].min()
    distmrt_min.append(temp)
    temp = hdb_features_df[k]['distanceWithMrt'].max()
    distmrt_max.append(temp)
    temp = hdb_features_df[k]['distanceWithRaffles'].min()
    distraffles_min.append(temp)
    temp = hdb_features_df[k]['distanceWithRaffles'].max()
    distraffles_max.append(temp)
    temp = hdb_features_df[k]['distanceWithGdPri'].min()
    distgdpri_min.append(temp)
    temp = hdb_features_df[k]['distanceWithGdPri'].max()
    distgdpri_max.append(temp)
    #flattype_min = hdb_features_df[k]['flat_type_num'].min()
    #flattype_max = hdb_features_df[k]['flat_type_num'].max()
    temp = hdb_features_df[k]['remaining_lease_num'].min()
    rlease_min.append(temp)
    temp = hdb_features_df[k]['remaining_lease_num'].max()
    rlease_max.append(temp)
    temp = hdb_features_df[k]['storey_range_num'].min()
    storeyrng_min.append(temp)
    temp = hdb_features_df[k]['storey_range_num'].max()
    storeyrng_max.append(temp)
    temp = hdb_features_df[k]['resale_price'].min()
    rprice_min.append(temp)
    temp = hdb_features_df[k]['resale_price'].max()
    rprice_max.append(temp)
    
    hdb_features_df[k]['flrarea_norm'] = hdb_features_df[k]['floor_area_sqm'].apply(normalization, args=(flrarea_min[k], flrarea_max[k]))
    hdb_features_df[k]['x_norm'] = hdb_features_df[k]['X'].apply(normalization, args=(x_min[k], x_max[k]))
    hdb_features_df[k]['y_norm'] = hdb_features_df[k]['Y'].apply(normalization, args=(y_min[k], y_max[k])) 
    hdb_features_df[k]['distmrt_norm'] = hdb_features_df[k]['distanceWithMrt'].apply(normalization, args=(distmrt_min[k], distmrt_max[k]))
    hdb_features_df[k]['distraffles_norm'] = hdb_features_df[k]['distanceWithRaffles'].apply(normalization, args=(distraffles_min[k], distraffles_max[k]))
    hdb_features_df[k]['distgdpri_norm'] = hdb_features_df[k]['distanceWithGdPri'].apply(normalization, args=(distgdpri_min[k], distgdpri_max[k]))
    #hdb_features_df[k]['flattype_norm'] = hdb_features_df[k]['flat_type_num'].apply(normalization, args=(flattype_min, flattype_max)) 
    hdb_features_df[k]['rlease_norm'] = hdb_features_df[k]['remaining_lease_num'].apply(normalization, args=(rlease_min[k], rlease_max[k]))
    hdb_features_df[k]['storeyrng_norm'] = hdb_features_df[k]['storey_range_num'].apply(normalization, args=(storeyrng_min[k], storeyrng_max[k]))
    hdb_features_df[k]['rprice_norm'] = hdb_features_df[k]['resale_price'].apply(normalization, args=(rprice_min[k], rprice_max[k])) 
    
    hdb_data_model_df.append(hdb_features_df[k].loc[:, hdb_features_df[k].columns.str.contains('norm')])
    
#hdb_data_model_df[5]

In [19]:
# Splitting train and test data for Linear Regression ML
hdb_predict_lr = []

for m in range(0, 6):
    X = hdb_data_model_df[m][['flrarea_norm', 
                              'x_norm', 
                              'y_norm', 
                              'distmrt_norm', 
                              'distraffles_norm', 
                              'distgdpri_norm', 
                              'rlease_norm', 
                              'storeyrng_norm']]
    y = hdb_data_model_df[m]['rprice_norm']

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=12) 
    print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    
    hdb_resale_price_lr = linear_model.LinearRegression()
    hdb_predict_lr.append(hdb_resale_price_lr)
    hdb_predict_lr[m].fit(X_train, y_train)

    #y_predicted = hdb_resale_price_lr.predict(X_test)
    y_predicted = hdb_predict_lr[m].predict(X_test)
    score = r2_score(y_test, y_predicted)

    print (str(flat_type[m]) + ", r2 score - " + str(score))

(738, 8) (246, 8) (738,) (246,)
2 ROOM, r2 score - 0.6551050597993556
(10365, 8) (3455, 8) (10365,) (3455,)
3 ROOM, r2 score - 0.6596588845394618
(17721, 8) (5908, 8) (17721,) (5908,)
4 ROOM, r2 score - 0.794369329112078
(10728, 8) (3576, 8) (10728,) (3576,)
5 ROOM, r2 score - 0.7664399526297156
(3442, 8) (1148, 8) (3442,) (1148,)
EXECUTIVE, r2 score - 0.6951137910069056
(23, 8) (8, 8) (23,) (8,)
MULTI-GENERATION, r2 score - 0.5880877800205326


In [28]:
print ("Flat Type (eg. 2 ROOM, 3 ROOM, 4 ROOM, 5 ROOM, EXECUTIVE, MULTI-GENERATION)")
flat_type_pdt = input()
print ("Floor Area (eg. 31sqm - 237sqm)")
floor_area_pdt = input()
print ("X-location")
x_loc_pdt = input()
print ("Y-location")
y_loc_pdt = input()
print ("Distance from Mrt")
dist_mrt_pdt = input()
print ("Distance from Raffles")
dist_raffles_pdt = input()
print ("Distance from Good Price")
dist_gdpri_pdt = input()
print ("Remaining Lease (0 to 99)")
remain_lease_pdt = input()
print ("Storey Range (eg. 01 TO 03, 04 TO 06, 07 TO 09, 10 TO 12, 13 TO 15, 16 TO 20)")
storey_range_pdt = input()

Flat Type (eg. 2 ROOM, 3 ROOM, 4 ROOM, 5 ROOM, EXECUTIVE, MULTI-GENERATION)
4 ROOM
Floor Area (eg. 31sqm - 237sqm)
85
X-location
39971
Y-location
34327
Distance from Mrt
600
Distance from Raffles
11000
Distance from Good Price
1040
Remaining Lease (0 to 99)
87
Storey Range (eg. 01 TO 03, 04 TO 06, 07 TO 09, 10 TO 12, 13 TO 15, 16 TO 20)
13 TO 15


In [29]:
flattype_numeric = convert_flattype(flat_type_pdt)
storey_range_numeric = convert_storeyrange(storey_range_pdt)

predict_flrarea_norm = normalization (float(floor_area_pdt), flrarea_min[flattype_numeric], flrarea_max[flattype_numeric])
predict_x_loc_norm = normalization (float(x_loc_pdt), x_min[flattype_numeric], x_max[flattype_numeric])
predict_y_loc_norm = normalization (float(y_loc_pdt), y_min[flattype_numeric], y_max[flattype_numeric])
predict_dist_mrt_norm = normalization (float(dist_mrt_pdt), distmrt_min[flattype_numeric], distmrt_max[flattype_numeric])
predict_dist_raffles_norm = normalization (float(dist_raffles_pdt), distraffles_min[flattype_numeric], distraffles_max[flattype_numeric])
predict_dist_gdpri_norm = normalization (float(dist_gdpri_pdt), distgdpri_min[flattype_numeric], distgdpri_max[flattype_numeric])
predict_remain_lease_norm = normalization (float(remain_lease_pdt), rlease_min[flattype_numeric], rlease_max[flattype_numeric])
predict_storey_range_norm = normalization (int(storey_range_numeric), storeyrng_min[flattype_numeric], storeyrng_max[flattype_numeric])

predict_features = [[predict_flrarea_norm,
                     predict_x_loc_norm,
                     predict_y_loc_norm,
                     predict_dist_mrt_norm,
                     predict_dist_raffles_norm,
                     predict_dist_gdpri_norm,
                     predict_remain_lease_norm,
                     predict_storey_range_norm]]

print(predict_features)

[[0.22058823529411764, 0.9147169517131496, 0.29768596666626007, 0.22130219823450623, 0.5246846277422105, 0.28549999363253303, 0.8150572831423895, 0.8]]


In [30]:
y_predict = hdb_predict_lr[flattype_numeric].predict(predict_features)

In [31]:
resale_price = y_predict*(rprice_max[flattype_numeric] - rprice_min[flattype_numeric]) + rprice_min[flattype_numeric]

print ("The predicted resale price - ", resale_price)

The predicted resale price -  [541064.88621283]
